# Data Cleaning & Dataset Integration with pandas (Marketing Analytics)

This notebook demonstrates a practical, repeatable workflow to **clean** messy tabular data and **combine** multiple sources into a single, analysis-ready dataset—exactly the kind of preparation work that powers reliable **campaign reporting**, **funnel analysis**, and **performance dashboards**.

**Tools:** Python, pandas  
**Core skills:** data quality checks, NULL handling, type casting, deduplication, aggregation, union (concat), joins (merge)


## Project goals

By the end of this notebook, you will have a **tidy, analysis-ready table** created from multiple raw files. Along the way, we:

- Improve data quality:
  - Handle missing values (NULLs)
  - Create calculated fields
  - Convert columns to `datetime`
  - Remove duplicate rows and validate row counts
- Combine datasets in the ways analysts use most often:
  - Aggregate with `groupby()` for KPIs
  - UNION tables with `concat()` (stacking similar datasets)
  - JOIN tables with `merge()` (connecting keys across sources)
- Produce a small set of **marketing-friendly outputs** (clean tables + summary metrics) that are ready for visualization or modeling.


## Import pandas
---
First things first, include your imports at the top of your notebook.


In [ ]:
import pandas as pd

## Load the data
---

Start by reading in the orders csv file in the datasets folder. We'll use the orders table from Super Store for the first set of tasks - you're already familiar with this dataset from working in SQL, now let's analyze it in Python!


In [ ]:
orders = pd.read_csv('orders.csv')

### Explore the dataset
---
Start with some exploratory analysis methods to inspect the data. Which of the methods we've learned so far should we always remember to use when working with a new dataset?


In [ ]:
orders.head()

In [ ]:
orders.info()

## Data Cleaning
---


Data cleaning is the process of assembling data into a usable format for analysis.

Common data cleaning actions include: 
- Reformatting dates so that Python recognizes them.
- Extracting day/hour/month/year from a date to aggregate by those categories.
- Removing duplicate values or rows.
- Combining data sources into one table.
- Concatenating or separating data.



The data sets that we receive as analysts are often very messy, but there’s no need to be fazed by them. While we can’t possibly cover every single cleaning function in this course, there are tons of resources out there about cleaning functions.


### Four Primary Strategies for Handling NULLs
---

Finding missing data is the easy part! Determining what to do next is more complicated. Typically, we’re most interested in knowing why we are missing data. Once we know the “type of missingness” (i.e., the source or cause of missing data), we can proceed effectively. This is essential to deciding whether to delete incomplete values or fill them in and, if so, with what.

Recap: **A NULL value is any missing value in your data.**

One common way of conceptualizing a NULL value is thinking of it as “empty” — not zero, not the word “NULL,” but simply empty.


1. Using **external references**, find the true value of the missing data and fill it in using `df['column'].fillna(value)`.
2. **Fill with some value:** we have a few options here!
    - Impute (i.e., fill in) missing values with the mean, median, or some other calculated value. For example: `.df['column'].fillna(df['column'].mean())`
    - Fill with specific values: If you think you know what the values should be, you can replace a value with something else to standardize datasets using `df.replace(value_old, value_new)`
    - Fill with interpolated values: pandas has a `.interpolate()` method which will automatically calculate missing values based on linear calculations.
3. **Ignore them:** if the missing data wasn't relevant to your analysis, it's okay to leave the NULLs where they are. 
4. **Remove rows** containing NULL values: For some analyses, rows without information in an important column are entirely useless. Use `df.dropna(subset=['column'])` to remove them. _A general rule of thumb is to not remove more than 5% of your dataset using this method._

For the last two options listed here, proceed with caution! These can rely on dangerous assumptions and are usually not good approaches!


We can use the `.isnull()` and `.sum()` functions to count the nulls in each column. Chain on the `.sort_values()` function to find the columns with the largest number of nulls.


In [ ]:
orders.isnull().sum().sort_values()

Looks like we have a small number of nulls in our region ID column. Let's drop these rows from our dataset.


In [ ]:
orders.dropna()

In [ ]:
orders.dropna(subset = ['region_id'], inplace = True)

Our IT team confirmed that all missing postal code values should be 10001. Let's fill the null values using a built-in pandas function called `.fillna()`. This function will fill all the null values with a specified replacement, and it accepts the `inplace = True` statement to make our changes stick.


In [ ]:
orders['postal_code'].isnull().sum()

In [ ]:
#orders['postal_code'].fillna(10001).isnull().sum()
orders['postal_code'].fillna(10001, inplace=True)

Printing out the sum of nulls in each column should now show that our dataset is null-free:


In [ ]:
orders['postal_code'].isnull().sum()

In [ ]:
orders['postal_code'].fillna(10001, inplace=True)

In [ ]:
orders['postal_code'].isnull().sum()

### Adding calculated values to our dataset
---

Many of our cleaning operations involve applying an operation to a Series. This caan be used to create new columns based on existing data.

For this example, let's write a profit_margin function that accepts a row of data, which is a dictionary. It should return the result of dividing the profit column by the sales column (i.e. profit/sales).

To start, we'll need to create a function that divides a profit value by a sum value for a single row of a DataFrame.


In [ ]:
def profit_margin(row):
    return row['profit'] / row['sales']

In [ ]:
orders.apply(profit_margin, axis = 1)

In [ ]:
orders['Kihoon'] = orders['profit'] / orders['sales']

In [ ]:
orders.tail(3)

Now we can apply this function directly to every row in our orders DataFrame at once using a nifty pandas function called `.apply()`. 

This will pass each row as an individual piece of input into our profit margin function and append the output of our function to a new column. 

This output will be whatever our function **returns**, so be careful not to use a print statement instead. The default output of a function that is missing a return statement will be a NoneType object, which will often appear as just the word "None".


Using `.apply()`, create a new column in the orders dataframe called 'profit_margin' by applying the profit margin function row-by-row. Run this using the parameter `axis=1` to apply a function to each row one at a time. Note that the error returned by setting `axis=0` is a KeyError.


In [ ]:
orders.head()

In [ ]:
orders.apply(profit_margin, axis = 1)

In [ ]:
orders['profit_margin'] = orders.apply(profit_margin, axis = 1)

In [ ]:
orders.head(3)

### Now you

We also want to create a column called `margin_categorization` that categorizes our new profit margin as "profitable", "unprofitable", or "break even". We can create a new function to do this, and then apply it to our orders DataFrame using the same method as above. 

The function should follow these rules:
- If the profit_margin is greater than 0, the function should return "profitable"
- If the profit_margin is 0, the function should return "break even"
- If the profit_margin is less than 0, the function should return "unprofitable"


In [ ]:
def margin_categorization(kihoon):
    if kihoon['profit_margin'] >0:
        return "profitable"
    elif kihoon['profit_margin'] == 0:
        return "break even"
    else:
        return "unprofitable"

In [ ]:
orders.apply(margin_categorization, axis=1)

Use the new `margin_category` column to find how many orders in the dataset were unprofitable. There are multiple ways to do this!


In [ ]:
orders['margin_category'] = orders.apply(margin_categorization, axis=1)
orders.head()

### Handling Dates and Times
---

Some of the most challenging, frequently ill-formatted types of data are dates and times. Fortunately, pandas is on top of it with the `.to_datetime()` method.

Once a Series has been given a datetime data type, we can use access methods to extract specific time properties, like day or hour.
We can also use the pandas `Timestamp()` method to convert data into timestamps:
`pd.Timestamp(date_string_or_number)`

Convert the ship_date column to a datetime object using `pd.to_datetime()`.


In [ ]:
orders.head()

In [ ]:
# THIS IS "WET" CODE, MAKE IT "DRY"

orders['ship_date'] = pd.to_datetime(orders['ship_date'], dayfirst=True)
orders['order_date'] = pd.to_datetime(orders['order_date'], dayfirst=True)

In [ ]:
orders.dtypes

In [ ]:
# stretch exercise - create a function to apply datetime conversion into all the related columns.
def makeDT(row):
    return row['order_date'] = pd.to_datetime(row['order_date'], dayfirst=True)


orders.apply(makeDT, )

### Addressing Duplicates
---

Fortunately, the issue of duplicate data is a mere pandas method away from being solved! We can use the `.drop_duplicates()` method.

If we want to drop duplicates based on certain columns, we can do that, too: `df.drop_duplicates(subset=['column_a', 'column_b'])`


In [ ]:
orders

In [ ]:
orders[['ship_mode', 'customer_id']]

In [ ]:
orders[['ship_mode', 'customer_id']].drop_duplicates()

In [ ]:
orders.drop_duplicates(subset=['ship_mode', 'customer_id'])[['product_id', 'sales']]

In [ ]:
orders.drop_duplicates()

## Combining data
---


### Unioning DataFrames
---

We can combine or concatenate two DataFrames together with the `pd.concat()` method. This gives us the option to stack the DataFrames vertically to add more rows, or add the DataFrames side by side as new columns. 

**Note:** There are a lot of parameters that can be used to control how datasets are unioned using `pd.concat()`, including whether you allow duplicate entries and whether you're concatenating axes that are not shared between the DataFrames. Be sure to read the documentation if you want to use this method!


In [ ]:
df1 = pd.DataFrame([['a', 1], ['b', 2]], columns=['letter', 'number'])
df2 = pd.DataFrame([['c', 3], ['d', 4]], columns=['letter', 'number'])

In [ ]:
df1

In [ ]:
df2

In [ ]:
# Gives more rows but note that index is repeated!
pd.concat([df1, df2], join = 'inner')

In [ ]:
pd.concat([df1, df2], join = 'outer')

In [ ]:
# Gives more columns but note that column names are repeated!
pd.concat([df1, df2], axis=1)

Now that we've seen a small example, let's apply this to the orders DataFrame.


In [ ]:
orders.dtypes

In [ ]:
# First, make small samples of the DataFrame that we can use for this example
orders_2016 = orders[orders['ship_date'].dt.year == 2016]
orders_2017 = orders[orders['ship_date'].dt.year == 2017]

In [ ]:
orders_2016.head()

In [ ]:
orders_2017.head()

In [ ]:
orders.shape, orders_2016.shape, orders_2017.shape

Use `pd.concat` by passing a **list of the dataframes you want to concatenate** as an argument. Save this concatenated version as a new DataFrame.


In [ ]:
df_list = [orders_2016, orders_2017]

df_concat = pd.concat(df_list)

In [ ]:
df_concat_2 = pd.concat(df_list, axis=1)
df_concat_2.head()

In [ ]:
df_concat

Note that the index values above don't match the total number of rows printed at the bottom of our newly unioned DataFrame! When you concatenate or union two DataFrames together, you'll need to reset the index if you want it to be cleanly number from 0 to the maximum number of rows.


In [ ]:
df_concat.reset_index()

### Joining DataFrames
---

JOINing is the process of combining DataFrames according to specific values. Traditionally, this would be done with SQL - we already know how to do this!

JOINing allows us to:
- Reduce the size of a database.
- Increase the speed at which data is queried and returned.
- Reduce the redundancy of the data stored in the database.
- Access data that is split across multiple tables

**Recap:** A JOIN relies on multiple data sets that share a common unique identifier, or "key".

JOIN is used to combine tables for the purpose of adding selection criteria and possibly additional columns. 
- They connect data sources together in order to use information from both tables to display a desired result.
- A JOIN allows for tables to be connected using common columns, which serve as unique identifiers (called KEYS).
- Note that unique identifiers aren’t required, but are almost always used to avoid unintended behavior.

The robust method for JOINing in pandas is `merge()`, which accepts several parameters:

`pd.merge(left_df, right_df, how, left_on, right_on)`

As you may have guessed, the first two parameters are the DataFrames to JOIN. The third parameter describes the type of JOIN, typically “left.” The last two parameters provide the column name for the shared column, or foreign key, that will be used to combine the two DataFrames.


Let's practice joining datasets by loading in the products, returns, and regions csv files from the datasets folder. These will all be familiar from our time spent in pgAdmin!


In [ ]:
products = pd.read_csv('datasets/products.csv', encoding='unicode_escape')
returns = pd.read_csv('datasets/returns.csv')
regions = pd.read_csv('datasets/regions.csv')


We want to join the products and orders dataframes. Explore both dataframes to identify the common column between them. Use a left join to combine the tables in a dataframe named `orders_with_products`.

First, print a clean DataFrame that shows all of the column names and data types for the orders table. Do the same for the products table.


In [ ]:
orders.dtypes

In [ ]:
pd.DataFrame(orders.dtypes, columns=['DataTypes'])

In [ ]:
pd.DataFrame(products.dtypes, columns=['DataTypes'])

These tables both have a column called `product_id` that we can use as a key to join on.


In [ ]:
orders_with_products=pd.merge(left = products, right = orders, how='left', on= 'product_id')

In [ ]:
orders_with_products.head(3)

Next, use `pd.merge` again to join the returns table onto our table called `orders_with_products`.


In [ ]:
returns.dtypes

We can use the `order_id` column as the key to join on here.


In [ ]:
pd.merge(left=orders_with_products, right = returns, how ='left', on='order_id')

In [ ]:
orders_with_products_returns =pd.merge(left=orders_with_products, right = returns, how ='left', on='order_id')

In [ ]:
orders_with_products_returns.head(3)

### Now you:

Finally, join the region dataset to the combined dataframe above. Determine which column to use as a key and name your final DataFrame `combined_df`.


In [ ]:
combined_df = pd.merge(left=orders_with_products_returns, right=regions, how='left', on='region_id')
combined_df.head()

### Aggregating with groupby()
---
In pandas, `groupby()` statements allow us to segment our population to a specific subset and draw calculations based on those segment. A basic example looks like this:
`data_frame.groupby(['column_a']).count()`

We can think about a `groupby()` statement in three steps:
- Split: Separate our DataFrame into groups according to a specific attribute.
- Apply: Apply some function to the groups, like sum, count, or max.
- Combine: Put our DataFrame back together and check the output.


We can use the .agg() method to get multiple aggregate values: 

`df.groupby('col_a')['col_b'].agg(['count', 'mean', 'min', 'max'])`

The above command does the following:
- Takes our DataFrame, `df`
- Groups it by the values in `col_a` ("Split")
- Calculates the count, mean, minimum, and maximum of `col_b` ("Apply")
- And combines the results in a Series or DataFrame that's easy for us to digest ("Combine")

We can also groupby() multiple columns to drill down further:
`df.groupby(['first_column', 'second_column'])`


Let's run through a couple of examples of groupby using this dataset. 

Segment the orders DataFrame and explore aggregate values to answer the following questions


Which discount results in the highest mean order quantity?


In [ ]:
# First, group by discount and take the mean of the quantity column
# orders.groupby('discount')
# orders.groupby('discount')['quantity']
# orders.groupby('discount')['quantity'].count()
# orders.groupby('discount')['quantity'].min()
# orders.groupby('discount')['quantity'].max()
orders.groupby('discount')['quantity'].mean().head(10)

In [ ]:
# Chain on a sort_values command to sort in DESCENDING order
orders.groupby('discount')['quantity'].mean().sort_values(ascending=False).head(10)

In [ ]:
# Access the index to get the discount values sorted by largest mean quantity
orders.groupby('discount')['quantity'].mean().sort_values(ascending=False).index

In [ ]:
# Select the first discount value - this will be associated with the largest mean quantity!
orders.groupby('discount')['quantity'].mean().sort_values(ascending=False).index[0]

Chaining commands like shown above can answer complex questions with just one line of code. It can be hard to read when you're just getting started learning Python, so don't be afraid to break it out across cells and see what your output looks like each step of the way! Take advantage of how easy it is to index DataFrames, Series, lists, and so on with our square brackets []. 

Let's answer another question: Which product has the highest mean discount applied?


In [ ]:
# First, group by product ID and find the average discount
orders.groupby('product_id')['discount'].mean()

In [ ]:
# Chain on a sort_values command to sort in DESCENDING order
orders.groupby('product_id')['discount'].mean().sort_values(ascending=False)

In [ ]:
orders.groupby('product_id')['discount'].mean().sort_values(ascending=False)['OFF-AP-10002899']

In [ ]:
# Access the index to get the product ID sorted by largest mean discount
orders.groupby('product_id')['discount'].mean().sort_values(ascending=False).index

In [ ]:
# Finally, select the first product ID in this index - this will be associated with the largest mean discount!
orders.groupby('product_id')['discount'].mean().sort_values(ascending=False).index[0]

Finally, we can apply our group by method to the combined DataFrame `combined_df` we created when joining tables together. Let's use this to determine the salesperson who is generating the most profit.


In [ ]:
# Determine which salesperson is generating the most profit
candidate_cols = [c for c in combined_df.columns if any(k in c.lower() for k in ('salesperson','sales_rep','sales_person','employee','rep'))]
if not candidate_cols:
    raise KeyError(f"No salesperson-like column found in combined_df. Columns: {combined_df.columns.tolist()}")

sales_col = candidate_cols[0]
profit_by_sales = combined_df.groupby(sales_col)['profit'].sum().sort_values(ascending=False)

# Top salesperson and their total profit
top_salesperson = profit_by_sales.index[0]
print(top_salesperson, profit_by_sales.iloc[0])

# Optionally view top 5
print(profit_by_sales.head(5))

## Lesson Summary
---
Today, we:

- Used pandas to handle missing or problematic data values.
- Identified appropriate cleaning strategies for specific types of data.
- Used groupby() and JOIN statements to combine data with pandas.
- Created insights from data by splitting and combining data segments.
